In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import strftime
import numpy as np

import scrape_url_list as s
import acquire as a
import prepare as p

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

---

In [2]:
# links = s.get_cached_links()

In [3]:
# links = links[:-2]

---

In [4]:
# link_list = []
# for link in links:
#     response = requests.get(link, headers={"user-agent": "Codeup DS"})
#     link_list.append({'link' : link, 'status_code' : response.status_code})

In [5]:
# df = pd.DataFrame(link_list)

In [6]:
# df.head()

In [7]:
# df[df.status_code != 200]

#### Takeaways from acquire:
- The urls were fine when making requests, all returned status code of 200
- But when I tried to use Zach's acquire module to get the needed content, 5 links returned `404` errors
    - Still not sure exaxtly why, but must have to do with the scopes associated with my personal access token and auther permissions or someting
    - Here are the ones that didn't work:
        - `aikorea/awesome-rl/`
        - `JoseDeFreitas/awesome-youtubers#machine-learning`
        - `Developer-Y/cs-video-courses#artificial-intelligence`
        - `Vedenin/useful-java-links#ii-databases-search-engines-big-data-and-machine-learning`
        - `alirezadir/machine-learning-interview-enlightener/blob/main/README.md`

In [8]:
# df = pd.read_json('data.json')

In [9]:
# df.head()

In [10]:
# df.shape

In [11]:
# df.language.value_counts(dropna=False)

In [12]:
# df.loc[0 ,'readme_contents']

---

Get some new links

In [13]:
# response = requests.get('https://github.com/trending', headers={"user-agent": "Codeup DS"})
# soup = BeautifulSoup(response.text)

In [14]:
# len(soup.select('.Box-row'))

In [15]:
# [link.attrs['href'] for link in soup.select_one('.Box').select('a')]

In [16]:
# soup.select_one('.Box').select('a')

In [17]:
# soup.select('.Box-row')[0].select_one('.h3.lh-condensed').a.attrs['href']

In [18]:
# boxes = soup.select('.Box-row')

In [19]:
# len(boxes)

In [20]:
# link_list = []
# for box in boxes:
#     link = box.select_one('.h3.lh-condensed').a.attrs['href']
#     link_list.append(link)

In [21]:
# link_list

In [22]:
# there are only 25 links here that are classified as trending

---

Most forked

In [23]:
# url = 'https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=forks&type=Repositories'

In [24]:
# # get first page
# response = requests.get(url, headers={"user-agent": "Codeup DS"})
# soup = BeautifulSoup(response.text)

In [25]:
# len(soup.select('.repo-list-item'))

In [26]:
# cards = soup.select('.repo-list-item')

In [27]:
# len(cards)

In [28]:
# cards[0].select_one('.v-align-middle').attrs['href']

In [29]:
# link_list = []
# for card in cards:
#     link = card.select_one('.v-align-middle').attrs['href']
#     link_list.append(link)

In [30]:
# link_list

In [31]:
# # get 10 pages
# url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'

In [32]:
# url = url.format(1)

In [33]:
# url

In [34]:
# response = requests.get(url, headers={"user-agent": "Codeup DS"})

In [35]:
# response

In [36]:
# soup = BeautifulSoup(response.text)

In [37]:
# cards = soup.select('.repo-list-item')

In [38]:
# len(cards)

In [39]:
# link_list = []
# for i in range(1,10):
#     url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'
#     url = url.format(i)
#     response = requests.get(url, headers={"user-agent": "Codeup DS"})
#     soup = BeautifulSoup(response.text)
#     cards = soup.select('.repo-list-item')
#     for card in cards:
#         link = card.select_one('.v-align-middle').attrs['href']
#         link_list.append(link)
    

In [40]:
# # this is the function that I used to get the top 500 most forked repos on github
# from time import sleep
# cards_list = []
# link_list = []
# for i in range(1,51):
#     url = 'https://github.com/search?o=desc&p={}&q=stars%3A%3E1&s=forks&type=Repositories'
#     url = url.format(i)
#     response = requests.get(url, headers={"user-agent": "Codeup"})
#     print(response.status_code)
#     soup = BeautifulSoup(response.text)
#     cards = soup.select('.repo-list-item')
#     cards_list.append(cards)
#     #print(i)
#     #print(url)
#     #print(len(cards))
#     #print(len(cards_list))
#     for card in cards:
#         link = card.select_one('.v-align-middle').attrs['href']
#         link_list.append(link)
#     sleep(30)
# [link[1:] for link in link_list]

In [41]:
# len(link_list)

In [42]:
# link_list[0][1:]

In [43]:
# [link[1:] for link in link_list]

---

In [2]:
df = pd.read_json('data.json')

In [3]:
df.head()

,repo,language,readme_contents
0,jtleek/datasharing,None,How to share data with a statistician\n=======...
1,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...
2,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...
3,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www...."
4,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...


In [4]:
df.shape

(500, 3)

In [5]:
df.language.value_counts(dropna=False)

JavaScript          100
Java                 77
NaN                  60
Python               54
C++                  30
Jupyter Notebook     26
TypeScript           26
Go                   19
HTML                 16
C                    15
PHP                  12
Ruby                 11
Shell                 8
Kotlin                5
CSS                   5
Swift                 4
C#                    4
Vue                   3
Rascal                3
Nix                   2
R                     2
Dart                  2
Objective-C           2
Scala                 2
Dockerfile            1
TeX                   1
Less                  1
PowerShell            1
Groovy                1
Vim script            1
Nunjucks              1
Assembly              1
SCSS                  1
ApacheConf            1
Rich Text Format      1
Rust                  1
Name: language, dtype: int64

In [6]:
df.shape

(500, 3)

In [7]:
df.isna().sum()

repo                0
language           60
readme_contents     0
dtype: int64

In [8]:
df.dropna().shape

(440, 3)

In [9]:
df = p.prep_nlp(df, original_text_col='readme_contents')

In [10]:
df.shape

(261, 6)

In [82]:
df.language.value_counts()

JavaScript          100
Java                 77
Python               54
C++                  30
TypeScript           26
Jupyter Notebook     26
Go                   19
HTML                 16
C                    15
PHP                  12
Ruby                 11
Shell                 8
Kotlin                5
CSS                   5
C#                    4
Swift                 4
Rascal                3
Vue                   3
R                     2
Nix                   2
Scala                 2
Objective-C           2
Dart                  2
TeX                   1
Dockerfile            1
Less                  1
Rust                  1
Nunjucks              1
PowerShell            1
Vim script            1
SCSS                  1
Groovy                1
ApacheConf            1
Assembly              1
Rich Text Format      1
Name: language, dtype: int64

In [83]:
# # need to lump all except top 4 as other
# top4 = ['JavaScript', 'Java', 'Python', 'C++']
# np.where((df['language'] if not in top4), 'Other', df['language'])
# # couldn't get npwhere to work with in operator

In [84]:
# df['county'] = df['fips'].apply(lambda x: 'Los Angeles' if x == 6037 else 'Orange' if x == 6059 else 'Ventura') # convert to county names

In [85]:
top4 = ['JavaScript', 'Java', 'Python', 'C++']
df.language = df.language.apply(lambda x: 'Other' if x not in top4 else x)

In [ ]:
# instead of lumping, let's just drop all others

In [86]:
df.language.value_counts()

Other         179
JavaScript    100
Java           77
Python         54
C++            30
Name: language, dtype: int64

In [92]:
df.shape

(440, 6)

In [94]:
df = df[df.language != 'Other']

In [95]:
df.shape

(261, 6)

In [87]:
def split_readme(df):
    '''
    This function takes in a df and splits it into train, validate, and test dfs
    final proportions will be 60/20/20 for train/validate/test
    '''
    train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.language)
    train, validate = train_test_split(train_validate, test_size=.25, random_state=123, stratify=train_validate.language)
    return train, validate, test

In [88]:
train, validate, text = p.split_readme(df)

In [89]:
train.shape, validate.shape, text.shape

((264, 6), (88, 6), (88, 6))

In [11]:
df = pd.read_json('data.json')

In [12]:
df.shape

(500, 3)

In [13]:
df.language.value_counts()

JavaScript          100
Java                 77
Python               54
C++                  30
TypeScript           26
Jupyter Notebook     26
Go                   19
HTML                 16
C                    15
PHP                  12
Ruby                 11
Shell                 8
CSS                   5
Kotlin                5
C#                    4
Swift                 4
Rascal                3
Vue                   3
R                     2
Nix                   2
Dart                  2
Objective-C           2
Scala                 2
ApacheConf            1
Vim script            1
TeX                   1
Groovy                1
PowerShell            1
Less                  1
Dockerfile            1
Rich Text Format      1
Nunjucks              1
Assembly              1
SCSS                  1
Rust                  1
Name: language, dtype: int64

In [14]:
df = p.prep_nlp(df, original_text_col='readme_contents')

In [16]:
df.language.value_counts()

JavaScript    100
Java           77
Python         54
C++            30
Name: language, dtype: int64

In [17]:
train, validate, test = p.split_readme(df)

Train Proportion: 0.60 (156 rows)
Validate Proportion: 0.20 (52 rows)    
Test Proportion: 0.20 (53 rows)
